In [6]:
from rdflib import Graph, ConjunctiveGraph, RDF, RDFS, OWL
from SPARQLWrapper import SPARQLWrapper2
import pandas as pds
import seaborn as sns

In [7]:
q = "select ?s ?p ?o where { ?s ?p ?o } limit 10"
sparql = SPARQLWrapper2("http://192.168.1.243:7200/repositories/EDR_NO_ES")
sparql.setQuery(q)

In [12]:
res = sparql.query().bindings
for r in res:
    print(r)

{'s': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'p': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'o': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property')}
{'s': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'p': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'o': Value(uri:'http://www.w3.org/2002/07/owl#Thing')}
{'s': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'p': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'o': Value(uri:'http://www.w3.org/2002/07/owl#ObjectProperty')}
{'s': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'), 'p': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'o': Value(uri:'http://www.w3.org/2000/01/rdf-schema#Class')}
{'s': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'), 'p': Value(uri:'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 'o': Value(uri:'http://www.w3.org/2002/07/owl#Class')}
{'s': Valu

In [17]:
for r in res:
    print('%s %s %s' % (r["s"].value, r["p"].value, r["o"].value))

http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
http://www.w3.org/1999/02/22-rdf-syntax-ns#Property http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://www.w3.org/1999/02/22-rdf-syntax-ns#Property http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
http://www.w3.org/1999/02/22-rdf-syntax-ns#Property http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://www.w3.org/2000/01/rdf-schema#Class http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://www.w3.org/2000/01/rdf-